In [1]:
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import numpy as np

from jaxlayerlumos import stackrt_eps_mu
from jaxlayerlumos import utils_materials

In [ ]:
# --- Demo configuration ---
freq_range = (0.2, 2.0)  # GHz
inc_angle = 0.0          # degrees

# Frequency grid: from 0.2 GHz to 2.0 GHz (converted to Hz)
frequencies = jnp.linspace(freq_range[0] * 1e9, freq_range[1] * 1e9, 100)
freq_ghz = np.array(frequencies) / 1e9  # for plotting if needed

# Define a sample design:
# Materials: starting with "Air", then five internal layers, and ending with "PEC"
design = {
    "materials_data": ["14", "8", "4", "4", "4"],
    "thicknesses_mm": [0.966, 1.002, 1.182, 0.984, 1.380],
}

# Create the full materials stack (boundary layers fixed)
materials = ["Air"] + design["materials_data"] + ["PEC"]

# Convert thicknesses from mm to m and add boundaries (set to 0)
thicknesses = jnp.array([0.0] + [x * 1e-3 for x in design["thicknesses_mm"]] + [0.0])
print("Initial thicknesses (m):", thicknesses)

# Get the effective permittivity and permeability for the stack.
eps_stack, mu_stack = utils_materials.get_eps_mu(materials, frequencies)

# Define an objective function that returns the maximum reflection in dB.
# We want to see how the maximum reflection responds to changes in the internal thicknesses.
def objective(thicknesses_internal):
    # Reconstruct the full thickness vector (fixing boundaries at 0)
    thicknesses_full = jnp.concatenate([jnp.array([0.0]), thicknesses_internal, jnp.array([0.0])])
    
    # Compute reflection coefficients at normal incidence.
    R_TE, _, R_TM, _ = stackrt_eps_mu(eps_stack, mu_stack, thicknesses_full, frequencies, inc_angle)
    R_linear = (R_TE + R_TM) / 2.0
    
    # Convert reflection to dB (with safeguard to avoid log10(0))
    R_db = 10 * jnp.log10(jnp.maximum(R_linear, 1e-12)).squeeze()
    
    # Return the maximum reflection in dB over the frequency range
    return jnp.max(R_db)

# Extract only the internal thicknesses (those that can be changed)
thicknesses_internal = thicknesses[1:-1]

Initial thicknesses (m): [0.       0.00018  0.000738 0.000918 0.000984 0.000114 0.      ]


In [16]:
# Compute the gradient of the objective with respect to the internal thicknesses.
grad_objective = jax.grad(objective)(thicknesses_internal)

# Print the gradient information
print("\nGradient of maximum reflection (in dB) with respect to each internal thickness:")
print(grad_objective)



Gradient of maximum reflection (in dB) with respect to each internal thickness:
[ 1.08564531e+02  3.60794514e-11 -5.83546353e-12 -3.01575956e-13
 -4.09332394e-13]


### Explanation:

The printed gradient values indicate how sensitive the maximum reflection is to changes in each of the
internal layer thicknesses. A negative gradient value means that increasing the thickness of that layer
will reduce the maximum reflection (in dB), whereas a positive value indicates the opposite effect.